In [1]:
# camera
from picamera import PiCamera
# image processing
from PIL import Image
import numpy as np
# sleep() now()
from time import sleep
import datetime
# for removing videos
import os
import cv2
from skimage.metrics import structural_similarity as ssim

camera = PiCamera()
camera.resolution = (360, 280)
original = cv2.imread('original.jpg')
videoList = []

def customerExists(org, thres):
    sleep(1)
    
    now = datetime.datetime.now()  # 현재 시간 저장
    filename =  now.strftime('%Y-%m-%d %H:%M:%S') # 파일 이름 설정(현재 시간 기반)
    
    camera.capture("./record/" + filename + ".jpg") # 현재 사진 촬영
    img = cv2.imread("./record/" + filename + ".jpg")
        
    grayO = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    grayI = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 원본 사진과 현재 사진 행렬 값 차이 계산
    diff = cv2.subtract(original, img)    
    (score, diff) = ssim(grayO, grayI, full=True)
    diff = (diff * 255).astype("uint8")   

    if int(score*100) < thres: # if customer exists
        print(np.sum(diff))
        camera.start_recording(output = "./record/" + filename + ".h264")
        sleep(20)
        camera.stop_recording()
        
        videoList.append(filename)
        return True
    else:
        videoList.append(filename)
        return False

In [2]:
# 폴더 생성 함수
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [3]:
# 재고 비교 함수
import pandas as pd

def cmpInven():
    df_store = pd.read_csv("inventory/inventory_store.csv")
    df_weight = pd.read_csv("inventory/inventory_weight.csv")
    
    df_store = df_store.set_index('상품명')
    df_weight = df_weight.set_index('상품명')
    
    count1 = int(df_store.loc['바나나우유'][0]) 
    count2 = int(df_weight.loc['바나나우유'][0]) 
    
    print(count1-count2)
    
    if count1 == count2:  # 무게 측정 재고와 결재 재고가 같은 경우
        return 0
    else:                        # 무게 측정 재고와 결재 재고가 다른 경우
        return (count1 - count2)

In [4]:
# 재고 오차 수정 함수
def correctInven():
    df_store = pd.read_csv("inventory/inventory_store.csv")
    df_weight = pd.read_csv("inventory/inventory_weight.csv")
    
    df_store = df_store.set_index('상품명')
    df_weight = df_weight.set_index('상품명')
    
    count1 = int(df_store.loc['바나나우유'][0]) 
    count2 = int(df_weight.loc['바나나우유'][0]) 
    
    time = str(datetime.datetime.now())
    head = '\n------' + time + '------\n'
    tail = '\n------------end------------\n'
    message = head + "밥그릇" + str(count1-count2) + "개 도난 발생\n" "weight_csv 기준으로 조율" + tail
    
    df_store.loc['바나나우유'][0] = df_weight.loc['바나나우유'][0] 
    df_store.to_csv("inventory/inventory_store.csv")
    
    
    # 누적 로그 기록
    log = open('event.txt', mode = 'at', encoding = 'utf-8')
    log.write(message)
    log.close()
    
    # 푸시 로그 기록
    path = './issue_data/' + time
    createFolder(path)
    message = "밥그릇" + str(count1-count2) + "개 도난 발생"
    log = open(path + '/content.txt', mode = 'wt', encoding = 'utf-8')
    log.write(message)
    log.close()
    
    # 푸시 전송할 사진 경로 변경
    src = './record/' + videoList[-1] + '.jpg'
    des = path + '/' + videoList[-1] + '.jpg'
    os.replace(src, des)

In [5]:
while(1):
    if customerExists(original, 65) : # 사람이 있는 경우
        print(22)
        continue
    else: # 사람이 없는 경우
        if cmpInven() != 0:  # 결재 재고와 무게 재고가 다른 경우 (도난 발생)
            count = cmpInven()
            correctInven()
            videoList = []
            print("밥그릇" + str(count)+"개 도난")
        else:  # 결재 재고와 무게 재고가 같은 경우
            print(videoList)
            for video in videoList:
                if os.path.isfile("./record/" + video + ".h264"):
                    os.remove("./record/" + video + ".h264")
                os.remove("./record/" + video + ".jpg")
            videoList = []
            print(11)
            continue
            

0
['2021-12-06 14:22:32']
11
0
['2021-12-06 14:22:34']
11
0
['2021-12-06 14:22:35']
11
0
['2021-12-06 14:22:37']
11
0
['2021-12-06 14:22:38']
11
0
['2021-12-06 14:22:40']
11
0
['2021-12-06 14:22:41']
11
0
['2021-12-06 14:22:43']
11
0
['2021-12-06 14:22:44']
11
0
['2021-12-06 14:22:46']
11
0
['2021-12-06 14:22:47']
11
0
['2021-12-06 14:22:49']
11
0
['2021-12-06 14:22:50']
11
0
['2021-12-06 14:22:52']
11
0
['2021-12-06 14:22:53']
11
0
['2021-12-06 14:22:55']
11
0
['2021-12-06 14:22:56']
11
0
['2021-12-06 14:22:58']
11
0
['2021-12-06 14:22:59']
11
0
['2021-12-06 14:23:01']
11
0
['2021-12-06 14:23:02']
11
0
['2021-12-06 14:23:04']
11
0
['2021-12-06 14:23:05']
11
0
['2021-12-06 14:23:07']
11
0
['2021-12-06 14:23:08']
11
0
['2021-12-06 14:23:10']
11
0
['2021-12-06 14:23:11']
11
0
['2021-12-06 14:23:13']
11
0
['2021-12-06 14:23:14']
11
0
['2021-12-06 14:23:16']
11
0
['2021-12-06 14:23:17']
11
0
['2021-12-06 14:23:19']
11
0
['2021-12-06 14:23:20']
11
0
['2021-12-06 14:23:22']
11
0
['2021-12-06

KeyboardInterrupt: 